In [1]:
%cd ../src
%pwd
%matplotlib inline
%load_ext autoreload
%autoreload 2

E:\Projecten\Sepsis\src


In [2]:
# general imports
import os
import pandas as pd
import matplotlib.pyplot as plt
import string
import datetime as dt

# utils import
from utils.files import get_latest_version
from utils.cross_validation import cross_validate_ROC, cross_validate_risk_ROC, compare_models, cross_validate_calibration
from utils.risk_scores import read_data_risk_score
from xgboost2.cross_validate import read_data

# xgboost
from xgboost import XGBClassifier

from sklearn.model_selection import GridSearchCV, cross_val_score

In [3]:
# model 1
model = 1
model_version_mort = 22

data_dict = read_data(model = model, 
                      out = 3, # mortality
                      version = model_version_mort)

2020-06-25 11:32:28,079 - root - INFO - cross_validate : 48 - Reading data from file: model_1_out_3_maindb.csv


--- Logging error ---
Traceback (most recent call last):
  File "c:\programdata\anaconda3\envs\ml_main\lib\logging\handlers.py", line 69, in emit
    self.doRollover()
  File "c:\programdata\anaconda3\envs\ml_main\lib\logging\handlers.py", line 393, in doRollover
    self.rotate(self.baseFilename, dfn)
  File "c:\programdata\anaconda3\envs\ml_main\lib\logging\handlers.py", line 110, in rotate
    os.rename(source, dest)
PermissionError: [WinError 32] Het proces heeft geen toegang tot het bestand omdat het door een ander
proces wordt gebruikt: 'E:\\Projecten\\Sepsis\\src\\logging\\scripts.log' -> 'E:\\Projecten\\Sepsis\\src\\logging\\scripts.log.2020-06-24_14'
Call stack:
  File "c:\programdata\anaconda3\envs\ml_main\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "c:\programdata\anaconda3\envs\ml_main\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "c:\programdata\anaconda3\envs\ml_main\lib\site-packages\ipykernel_launcher.py", l

2020-06-25 11:32:29,239 - root - INFO - cross_validate : 63 - Selecting Y-value
2020-06-25 11:32:29,242 - root - INFO - cross_validate : 66 - Selecting X-values


--- Logging error ---
Traceback (most recent call last):
  File "c:\programdata\anaconda3\envs\ml_main\lib\logging\handlers.py", line 69, in emit
    self.doRollover()
  File "c:\programdata\anaconda3\envs\ml_main\lib\logging\handlers.py", line 393, in doRollover
    self.rotate(self.baseFilename, dfn)
  File "c:\programdata\anaconda3\envs\ml_main\lib\logging\handlers.py", line 110, in rotate
    os.rename(source, dest)
PermissionError: [WinError 32] Het proces heeft geen toegang tot het bestand omdat het door een ander
proces wordt gebruikt: 'E:\\Projecten\\Sepsis\\src\\logging\\scripts.log' -> 'E:\\Projecten\\Sepsis\\src\\logging\\scripts.log.2020-06-24_14'
Call stack:
  File "c:\programdata\anaconda3\envs\ml_main\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "c:\programdata\anaconda3\envs\ml_main\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "c:\programdata\anaconda3\envs\ml_main\lib\site-packages\ipykernel_launcher.py", l

In [4]:
from itertools import product
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_curve, auc
import numpy as np
from math import sqrt

max_depth = [3, 5, 7, 9, 11, 13, 15]
estimators = [100, 150, 200, 250, 300, 350]
combinations = product(max_depth, estimators, repeat=1)

cv = StratifiedKFold(n_splits=5)
aucs = {}

x = data_dict['x']
y = data_dict['y']

for c in combinations:
    _aucs = []
    model = XGBClassifier(max_depth=c[0], 
                          n_estimators=c[1])
    
    for train, test in cv.split(x, y):
        model.fit(x[train], y[train])
        
        probas_ = model.predict_proba(x[test])
        fpr, tpr, thresholds = roc_curve(y[test], probas_[:, 1])
        _aucs.append(auc(fpr, tpr))
        
    mean_auc = np.mean(_aucs)
    _ci = 1.96 * (np.std(_aucs) / sqrt(5))
    aucs[c] = "{:.3f} ({:.3f}-{:.3f})".format(mean_auc,
                                             mean_auc - _ci,
                                             mean_auc + _ci)
    print(aucs)

{(3, 100): '0.819 (0.801-0.838)'}
{(3, 100): '0.819 (0.801-0.838)', (3, 150): '0.816 (0.800-0.832)'}
{(3, 100): '0.819 (0.801-0.838)', (3, 150): '0.816 (0.800-0.832)', (3, 200): '0.815 (0.799-0.831)'}
{(3, 100): '0.819 (0.801-0.838)', (3, 150): '0.816 (0.800-0.832)', (3, 200): '0.815 (0.799-0.831)', (3, 250): '0.816 (0.799-0.832)'}
{(3, 100): '0.819 (0.801-0.838)', (3, 150): '0.816 (0.800-0.832)', (3, 200): '0.815 (0.799-0.831)', (3, 250): '0.816 (0.799-0.832)', (3, 300): '0.814 (0.797-0.832)'}
{(3, 100): '0.819 (0.801-0.838)', (3, 150): '0.816 (0.800-0.832)', (3, 200): '0.815 (0.799-0.831)', (3, 250): '0.816 (0.799-0.832)', (3, 300): '0.814 (0.797-0.832)', (3, 350): '0.814 (0.797-0.832)'}
{(3, 100): '0.819 (0.801-0.838)', (3, 150): '0.816 (0.800-0.832)', (3, 200): '0.815 (0.799-0.831)', (3, 250): '0.816 (0.799-0.832)', (3, 300): '0.814 (0.797-0.832)', (3, 350): '0.814 (0.797-0.832)', (5, 100): '0.803 (0.777-0.828)'}
{(3, 100): '0.819 (0.801-0.838)', (3, 150): '0.816 (0.800-0.832)', (3

In [5]:
# model 1
model = 3
model_version_mort = 22

data_dict = read_data(model = model, 
                      out = 3, # mortality
                      version = model_version_mort)

2020-06-25 12:38:57,162 - root - INFO - cross_validate : 48 - Reading data from file: model_3_out_3_maindb.csv


--- Logging error ---
Traceback (most recent call last):
  File "c:\programdata\anaconda3\envs\ml_main\lib\logging\handlers.py", line 69, in emit
    self.doRollover()
  File "c:\programdata\anaconda3\envs\ml_main\lib\logging\handlers.py", line 393, in doRollover
    self.rotate(self.baseFilename, dfn)
  File "c:\programdata\anaconda3\envs\ml_main\lib\logging\handlers.py", line 110, in rotate
    os.rename(source, dest)
PermissionError: [WinError 32] Het proces heeft geen toegang tot het bestand omdat het door een ander
proces wordt gebruikt: 'E:\\Projecten\\Sepsis\\src\\logging\\scripts.log' -> 'E:\\Projecten\\Sepsis\\src\\logging\\scripts.log.2020-06-24_14'
Call stack:
  File "c:\programdata\anaconda3\envs\ml_main\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "c:\programdata\anaconda3\envs\ml_main\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "c:\programdata\anaconda3\envs\ml_main\lib\site-packages\ipykernel_launcher.py", l

2020-06-25 12:38:58,355 - root - INFO - cross_validate : 63 - Selecting Y-value
2020-06-25 12:38:58,355 - root - INFO - cross_validate : 66 - Selecting X-values


--- Logging error ---
Traceback (most recent call last):
  File "c:\programdata\anaconda3\envs\ml_main\lib\logging\handlers.py", line 69, in emit
    self.doRollover()
  File "c:\programdata\anaconda3\envs\ml_main\lib\logging\handlers.py", line 393, in doRollover
    self.rotate(self.baseFilename, dfn)
  File "c:\programdata\anaconda3\envs\ml_main\lib\logging\handlers.py", line 110, in rotate
    os.rename(source, dest)
PermissionError: [WinError 32] Het proces heeft geen toegang tot het bestand omdat het door een ander
proces wordt gebruikt: 'E:\\Projecten\\Sepsis\\src\\logging\\scripts.log' -> 'E:\\Projecten\\Sepsis\\src\\logging\\scripts.log.2020-06-24_14'
Call stack:
  File "c:\programdata\anaconda3\envs\ml_main\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "c:\programdata\anaconda3\envs\ml_main\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "c:\programdata\anaconda3\envs\ml_main\lib\site-packages\ipykernel_launcher.py", l

In [6]:
from itertools import product
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_curve, auc
import numpy as np
from math import sqrt

max_depth = [3, 5, 7, 9, 11, 13, 15]
estimators = [100, 150, 200, 250, 300, 350]
combinations = product(max_depth, estimators, repeat=1)

cv = StratifiedKFold(n_splits=5)
aucs = {}

x = data_dict['x']
y = data_dict['y']

for c in combinations:
    _aucs = []
    model = XGBClassifier(max_depth=c[0], 
                          n_estimators=c[1])
    
    for train, test in cv.split(x, y):
        model.fit(x[train], y[train])
        
        probas_ = model.predict_proba(x[test])
        fpr, tpr, thresholds = roc_curve(y[test], probas_[:, 1])
        _aucs.append(auc(fpr, tpr))
        
    mean_auc = np.mean(_aucs)
    _ci = 1.96 * (np.std(_aucs) / sqrt(5))
    aucs[c] = "{:.3f} ({:.3f}-{:.3f})".format(mean_auc,
                                             mean_auc - _ci,
                                             mean_auc + _ci)
    print(aucs)

{(3, 100): '0.838 (0.806-0.871)'}
{(3, 100): '0.838 (0.806-0.871)', (3, 150): '0.834 (0.805-0.864)'}
{(3, 100): '0.838 (0.806-0.871)', (3, 150): '0.834 (0.805-0.864)', (3, 200): '0.831 (0.804-0.858)'}
{(3, 100): '0.838 (0.806-0.871)', (3, 150): '0.834 (0.805-0.864)', (3, 200): '0.831 (0.804-0.858)', (3, 250): '0.827 (0.801-0.852)'}
{(3, 100): '0.838 (0.806-0.871)', (3, 150): '0.834 (0.805-0.864)', (3, 200): '0.831 (0.804-0.858)', (3, 250): '0.827 (0.801-0.852)', (3, 300): '0.824 (0.800-0.849)'}
{(3, 100): '0.838 (0.806-0.871)', (3, 150): '0.834 (0.805-0.864)', (3, 200): '0.831 (0.804-0.858)', (3, 250): '0.827 (0.801-0.852)', (3, 300): '0.824 (0.800-0.849)', (3, 350): '0.823 (0.799-0.848)'}
{(3, 100): '0.838 (0.806-0.871)', (3, 150): '0.834 (0.805-0.864)', (3, 200): '0.831 (0.804-0.858)', (3, 250): '0.827 (0.801-0.852)', (3, 300): '0.824 (0.800-0.849)', (3, 350): '0.823 (0.799-0.848)', (5, 100): '0.833 (0.805-0.860)'}
{(3, 100): '0.838 (0.806-0.871)', (3, 150): '0.834 (0.805-0.864)', (3